In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Flatten,Conv1D,MaxPooling1D,Dropout
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zipfile
from numpy import array
from numpy import hstack

In [ ]:
#zip = zipfile.ZipFile('combined_df_1030.zip')
#zip.extractall()

In [ ]:
#zip = zipfile.ZipFile('combined_df_2030.zip')
#zip.extractall()

In [ ]:
#zip = zipfile.ZipFile('combined_df_3030.zip')
#zip.extractall()

In [4]:
"""
blocks = [1030, 2030, 3030]
#blocks = [1030, 2030, 3030, 4030, 5030, 6030, 7030]

dfss = []
for block in blocks:
    print(block)
    dfss.append(pd.read_csv('combined_df_'+str(block)+'.csv'))
df = pd.concat(dfss) 
"""

"\nblocks = [1030, 2030, 3030]\n#blocks = [1030, 2030, 3030, 4030, 5030, 6030, 7030]\n\ndfss = []\nfor block in blocks:\n    print(block)\n    dfss.append(pd.read_csv('combined_df_'+str(block)+'.csv'))\n    \n"

In [29]:
print('Enter 1 for single file training, 2 for blockwise training, 3 for full-blisk training')

def single():
    dir_final_save  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Combined_Data/'
    print('Input block number and angle')
    block = input()
    angle = input()
    df = pd.read_excel(str(dir_final_save)+'finaldf_forward_with_compensation'+str(block)+'__'+str(int(angle))+'.xlsx')
    print(df.shape)
    return df

def block():
    dir_block_save  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Combined_Data/Blockwise/'
    print('Input block number')
    block = input()
    df = pd.read_csv(str(dir_block_save)+str(block)+'.csv', mode='a', chunksize='10000',index=False)
    print(df.shape)
    return df
    
def blisk():
    dir_blisk_save  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Combined_Data/Blisk/'
    df = pd.read_csv(str(dir_blisk_save)+'cleaned_full_blisk'+'.csv', mode='a', chunksize='10000',index=False)
    print(df.shape)
    return df
    
switch = { 
           1 : single,
           2 : block,
           3 : blisk,
          }

df = switch[int(input())]()

Enter 1 for single file training, 2 for blockwise training, 3 for full-blisk training
1
Input block number and angle
1030
0
(21571, 69)


In [30]:
df.columns

Index(['Leading angle', 'Side tilt angle', 'Surface Contact point X',
       'Surface contact point Y', 'Surface contact point z',
       'Surface Orietation X', 'Surface Orietation Y', 'Surface Orietation Z',
       'Tool Tip Point X', 'Tool Tip Point Y', 'Tool Tip Point Z',
       'Tool Orientation X', 'Tool Orientation Y', 'Tool Orientation Z',
       'Closes Point Distance (Surface Contact Point - Acquired TCP 1030 blade0)',
       'time', 'MachineX', 'MachineY', 'MachineZ', 'MachineA', 'MachineC',
       'LoadSpindle', 'TransfX', 'TransfY', 'TransfZ', 'TransfI', 'TransfJ',
       'TransfK', 'executionDuration', 'previousExecutionDuration',
       'relativeTimetoPreviousCall program number', 'timeStamp', 'Tool Length',
       'Tool Radius', 'G54-X', 'G54-Y', 'G54-Z', 'G54-A', 'G54-C',
       'ProgNumber', 'Blade', 'Fx_mean', 'Fy_mean', 'Fz_mean', 'Mz_mean',
       'Fc_mean', 'Fa_mean', 'FcN_mean', 'Fres_mean', 'Fx_sigma', 'Fy_sigma',
       'Fz_sigma', 'Mz_sigma', 'Fc_sigma', 'Fa_s

In [ ]:
def get_dataset():
    X = df[['Leading angle','Side tilt angle','Tool Tip Point X',
           'Tool Tip Point Y', 'Tool Tip Point Z', 'Tool Orientation X',
           'Tool Orientation Y', 'Tool Orientation Z',]].copy(deep=True).to_numpy()
    y = df[['MachineX', 'MachineY', 'MachineZ', 'MachineA', 'MachineC']].copy(deep=True).to_numpy()
    
    return X,y

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(30, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X[0:100]):
        print('Train index:', train_ix.shape,'Test_index',test_ix.shape)
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        
        # fit model
        print('model fit')
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        print('model evaluate')
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('mae:',mae)
        results.append(mae)
    return results
 
# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X[:,:], y[:,:])
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

In [ ]:
np.sqrt(18.504)

In [ ]:
"""
from numpy import asarray
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
 
# get the dataset
def get_dataset():
    X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3, random_state=2)
    return X, y
 
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
    model.compile(loss='mae', optimizer='adam')
    return model
 
# load dataset
X, y = get_dataset()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose=0, epochs=100)
# make a prediction for new data
row = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]
newX = asarray([row])
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])
"""

In [ ]:
!pip install tensorflow

In [ ]:
!pip install pandas

In [ ]:
!pip install sklearn

In [ ]:
!pip install h5py

In [ ]:
!pip install matplotlib

In [ ]:
!pip install PyAstronomy

In [ ]:
!pip install Scipy

In [ ]:
!pip install matplotlib

In [2]:
!pip install keras-tuner --upgrade

  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9562 sha256=036f26188624ff7af08946f306cc0f1e619081c68fe65b8cdd06b968731df154
  Stored in directory: c:\users\rohan\appdata\local\pip\cache\wheels\a8\67\9d\0dece28e14096b9ef4403317a090c3bfe1dc6a4d0a1287360a
Successfully built kt-legacy


In [ ]:
!pip install pydot

In [ ]:
!pip install graphviz

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from platform import python_version
print(python_version())